In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib oauth2client

In [ ]:
import json
import yaml
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

In [ ]:
SCOPES = ['https://www.googleapis.com/auth/drive']

In [ ]:
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    '/path/to/your/service_account_credentials_file.json', SCOPES)
credentials=credentials.create_delegated("user@domain.com")
drive3 = build('drive', 'v3', credentials=credentials)
drive2 = build('drive', 'v2', credentials=credentials)

In [ ]:
file_name="secret"
page_token = None
page=0
while page < 5:
    response = drive3.files().list(q=f"name contains '{file_name}'",
                                          corpora='allDrives',
                                          fields='nextPageToken, files(id, name)',
                                  includeItemsFromAllDrives=True,
                                  supportsAllDrives=True,
                                          pageToken=page_token).execute()
    for file in response.get('files', []):
        print('Found file: %s (%s)' % (file.get('name'), file.get('id')))
    page_token = response.get('nextPageToken', None)
    page+=1
    if page_token is None:
        break


In [ ]:
file_id='the-file-id-returned-above'

#driveV2 has much more metadata
metadata=drive2.files().get(fileId=file_id).execute()
print(yaml.dump(metadata))

In [ ]:
permissions = drive2.permissions().list(fileId=file_id, useDomainAdminAccess=False).execute()
perms=permissions.get('items', [])
for p in perms:
    print(p)

In [ ]:
permission_id='the-permission-id-returned-above'
drive2.permissions().delete(fileId=file_id, permissionId=permission_id).execute()